<!-- metadata: title -->
# Kenya Unit Trusts: Money Market Fund(KES) Analysis

<!-- metadata: subtitle -->
> ### Can Kenyan Money Market Funds gurantee capital preservation? 

<!-- metadata: date, type=date -->
**Published Date:**
2024-03-03

<!-- metadata: date-modified, type=date-->
**Date Modified:**
2024-05-05

<!-- metadata: keywords, type=array -->
**Keywords:**
  - money
  - kenya
  - unit-trusts
  - money-market-funds
  - MMF

<!-- metadata: categories, type=array -->
**Categories:**
  - kenya-unit-trusts
  - money-market-funds

## Description

<!-- metadata: description -->
Money market is a form of unit trust, where fund managers collect money from the group of investors, and invest on their behave. This reduces the overhead of managing your portfio and significantly reduces your risk. Let's statistically and critically analyze Money market funds in Kenya in general using publicly available information and hopefully paint a clearer picture of the state of unit trusts in Kenya. 

- What are the risk factors that exist? 
- What is the performance of money market funds?

## Abstract

<!-- metadata: abstract -->
Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.

## Introduction

In Kenya, a good number of the population have a bank account, even if not the traditional bank account. It is estimated that by year 2029, 99.9% of Kenyans will be banked, ^[Population share with banking account in Kenya 2014-2029
Published by J. Degenhard, Jan 30, 2024. <https://www.statista.com/forecasts/1149636/bank-account-penetration-forecast-in-kenya>]. With increase in financial access, financial literacy and regular individuals will want to venture in the teritory of finantial assets huunting for higher interest rates. One of the most attractive entry level high-yield financial asset is the unit trust, specifically the money market funds. Its easy to start, deposit, withdraw, and its interest is daily compunding, while offering higher interest than any bank.

## Analysis

### Imports

In [ ]:
import sys
import os

# Add parent directory to sys.path
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_dir)

%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pyppeteer.page import Page
import asyncio
from utils.web_screenshot import web_screenshot_async
import json
from utils.get_browser import get_browser_page_async

### Fund Managers

Let's start of by listing all the certified fund mangers in Kenya by CMA.^[Approved Fund Managers by CMA. <https://www.cma.or.ke/licensees-market-players/>]

Lets start with a screenshot of the webpage.

In [ ]:
async def action(page: Page):
    await page.waitForSelector('ul.module-accordion')
    elements = await page.querySelectorAll('li .accordion-title')
    # Iterate through the elements to find the one containing 'FUND MANAGERS'
    for element in elements:
        text_content = await page.evaluate('(element) => element.textContent', element)
        if 'FUND MANAGERS' in text_content:
            # Click on the target element
            await element.click()
            break
    else:
        print('Element not found')
    await page.waitForSelector('li.current.builder-accordion-active')
    await asyncio.sleep(1)

await web_screenshot_async(
    "https://www.cma.or.ke/licensees-market-players/", 
    action = action,
    width=1500)

Let's query the "Fund Managers" table.

In [ ]:
import io
from bs4 import BeautifulSoup
from urllib.request import urlopen

html_content = urlopen("https://www.cma.or.ke/licensees-market-players/").read()
fund_managers_html_table = BeautifulSoup(html_content, "html.parser")\
    .find('span', string="FUND MANAGERS")\
        .parent\
            .parent\
                .parent\
                    .find('table')

fund_managers_df = pd.read_html(io.StringIO(str(fund_managers_html_table)))[0].dropna()
fund_managers_df

The address of `African Alliance Kenya Asset Management Limited` doesnt seem to be valid, lets populate it with `P.O. Box 27639 Nairobi 00506`

In [ ]:
fund_managers_df.loc[fund_managers_df['LICENCE NO.'] == 165, 'ADDRESS'] = 'P.O. Box 27639 Nairobi 00506'
fund_managers_df

### Show a map of the fund managers

ssss

### Add head quarter location

sssss

### Getting the Perormance

According to Capital Markets Authority, fund managers are required to publish their yields daily in a reputable newspaper. What this means accessing hostorical records requires a significant investment in time to collect, aggregate and validate the published yields. That not withstanding, two very important questions become obvious:
 - what is a reputbale newspaper? 
 - what yield do they publish, gross or net?
 - What picture does an anualized daily rate paint?
 - Is interest deffered or carried forward (reporting a convervative figure and retaining the rest for rainy day to preserve a picture of good performance)?

Luckily, one of the fund managers, one that has recently found its self in the courts far more often than it would have wished, does exelent investment and market research. It has a good-enough aggregate of fund managers and their performance weekly and monthly. We are going to crawl their data for analysis, and analyze it for manipulation.If the data comeout clean, we will analyze the trend of fund managers using the data.

### Cytton Research

We are going to crawl the data from cytonn research, https://cytonn.com/researches/categories/1

#### screen shots

Lests start with a view of weekly reports

In [ ]:
await web_screenshot_async(
    "https://cytonn.com/researches/categories/1",
    width=1000)

Here is the latest report

In [ ]:
await web_screenshot_async(
    "https://cytonnreport.com/research/progress-of-retirement",
    width=1000)

Instead of directly crawling HTML from https://cytonn.com/researches/ page, we can instead crawl JSON from https://cytonnreport.com/research page, using the link https://cytonnreport.com/get/allreports.

In [ ]:
from pyppeteer.page import Request, Response

async def get_all_cytonn_reports():
    async def modify_json_request(request: Request):
        if request.resourceType == "fetch" and request.url == "https://cytonnreport.com/get/allreports":
            # Create a new request body
            modified_data = json.dumps({"pagination": {"per_page": 700}}).encode("utf-8")
            # Create a new request with the modified data
            await request.continue_(
                overwritePostData=True,
                postData=modified_data
            )
        else:
            # Continue other requests as usual
            await request.continue_()
    page, browser = await get_browser_page_async()
    # Enable request interception
    await page.setRequestInterception(True)
    page.on("request", lambda req: asyncio.ensure_future(modify_json_request(req)))
    # Navigate to the desired URL
    await page.goto("https://cytonnreport.com/research")
    # Wait for the AJAX request and get the response
    response: Response = await page.waitForResponse(lambda response: response.url == "https://cytonnreport.com/get/allreports")
    response_body = await response.json()
    await browser.close()
    return response_body

all_cytonn_reports = await get_all_cytonn_reports()
all_cytonn_reports

In [ ]:
[i for i in all_cytonn_reports['data']]

[Disclaimer:](https://cytonn.com/uploads/downloads/progress-of-retirement-benefit-schemes-in-kenya-2024-cytonn-weekly-212024-vf.pdf) The views expressed in this publication are those of the writers where particulars are not
warranted. This publication, which is in compliance with Section 2 of the Capital Markets Authority Act Cap
485A, is meant for general information only and is not a warranty, representation, advice, or solicitation of
any nature. Readers are advised in all circumstances to seek the advice of a registered investment advisor.